In [1]:
# health_monitoring/training/train_autoencoder_torch.ipynb
# PPG-DaLiA 데이터 사용
# PyTorch Autoencoder 학습 파이프라인
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os

print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [2]:
# 루트 경로
cwd = Path(os.getcwd())
ROOT_DIR = cwd.parent

print("루트 경로:", ROOT_DIR)

# 데이터 로드
data_path = ROOT_DIR / "training" / "all_windows.npy"
windows = np.load(data_path)

루트 경로: c:\Users\minsu\i-room-ai\health_monitoring


In [3]:
# Normalize 0~1
windows = (windows - windows.min()) / (windows.max() - windows.min())
windows = windows.astype(np.float32)

# Torch tensor
data_tensor = torch.tensor(windows)
dataset = TensorDataset(data_tensor)

In [4]:
# Train/val split
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda
